# Example Notebook

In [1]:
import json 
from pathlib import Path
import shutil

from wattile.entry_point import init_logging, create_input_dataframe, run_model
from wattile.data_reading import read_dataset_from_file
from wattile.buildings_processing import prep_for_rnn
from wattile.entry_point import run_model


PROJECT_DIRECTORY = Path().resolve().parents[1]


In [2]:
"""
For this example, we will be using the default configs.
Check out the docs for an explaination of each config.
"""
with open(PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r") as f:
    configs = json.load(f)

exp_dir = PROJECT_DIRECTORY / "notebooks" / "exp_dir"
if exp_dir.exists():
    shutil.rmtree(exp_dir)
exp_dir.mkdir()

configs["exp_dir"] = str(exp_dir)
configs["data_dir"] = str(PROJECT_DIRECTORY / "tests" / "data")

configs

{'building': 'Synthetic Site',
 'target_var': 'Synthetic Site Electricity Main Total Power',
 'start_time': '2018-01-01T00:00:00-07:00',
 'end_time': '2022-01-01T00:00:00-07:00',
 'data_time_interval_mins': 1,
 'predictor_columns': ['Synthetic Weather Station Dew Point Temperature',
  'Synthetic Weather Station Diffuse Horizontal Irradiance',
  'Synthetic Weather Station Direct Normal Irradiance',
  'Synthetic Weather Station Dry Bulb Temperature',
  'Synthetic Weather Station Global Horizontal Irradiance',
  'Synthetic Weather Station Relative Humidity',
  "Synthetic Weather Station Wind Speed"],
 'arch_version': 4,
 'exp_id': '7-9-21',
 'arch_type': 'RNN',
 'arch_type_variant': 'lstm',
 'transformation_method': 'minmaxscale',
 'train_batch_size': 26,
 'val_batch_size': 1,
 'convert_csvs': False,
 'exp_dir': '/Users/heslinge/Repos/Wattile/notebooks/exp_dir',
 'data_dir': '/Users/heslinge/Repos/Wattile/tests/data',
 'resample_freq': 15,
 'sequence_freq_min': 15,
 'splicer': {'active': 

In [3]:
"""
Firstly, we will read the raw data from the dataset. 
Checkout the docs for an indepth explaination of necessary dataset structure.
"""
data = read_dataset_from_file(configs)
data


,Synthetic Weather Station Dew Point Temperature,Synthetic Weather Station Diffuse Horizontal Irradiance,Synthetic Weather Station Direct Normal Irradiance,Synthetic Weather Station Dry Bulb Temperature,Synthetic Weather Station Global Horizontal Irradiance,Synthetic Weather Station Relative Humidity,Synthetic Weather Station Wind Speed,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,104.800003
2021-12-01 07:01:00+00:00,15.183906,-1.496226,0.207964,56.174000,-1.841866,17.879999,11.265539,96.650002
2021-12-01 07:02:00+00:00,14.891221,-1.441191,0.457520,55.886002,-1.873579,17.809999,12.777752,96.199997
2021-12-01 07:03:00+00:00,14.836164,-1.371754,0.473117,55.796001,-1.860084,17.820000,12.694983,95.599998
2021-12-01 07:04:00+00:00,14.782966,-1.317349,0.545903,55.723999,-1.843271,17.820000,11.632407,100.650002
...,...,...,...,...,...,...,...,...
2021-12-08 06:55:00+00:00,11.210565,-1.487212,-0.254757,42.285198,-1.814939,24.680000,0.000000,109.664803
2021-12-08 06:56:00+00:00,11.190062,-1.445182,-0.233960,41.997200,-1.852901,24.930000,0.000000,107.002800
2021-12-08 06:57:00+00:00,11.223961,-1.396302,-0.145575,41.669601,-1.841248,25.290001,3.545647,106.480400


In [4]:
"""
Next, we call `prep_for_rnn` do preform some data preprocessing.
"""
train_df, val_df = prep_for_rnn(configs, data)
train_df

,Synthetic Weather Station Dew Point Temperature_min_lag24,Synthetic Weather Station Diffuse Horizontal Irradiance_min_lag24,Synthetic Weather Station Direct Normal Irradiance_min_lag24,Synthetic Weather Station Dry Bulb Temperature_min_lag24,Synthetic Weather Station Global Horizontal Irradiance_min_lag24,Synthetic Weather Station Relative Humidity_min_lag24,Synthetic Weather Station Wind Speed_min_lag24,sin_HOD_min_lag24,cos_HOD_min_lag24,DOW_binary_reg_0_min_lag24,...,DOW_binary_reg_0_max,DOW_binary_reg_1_max,DOW_binary_reg_2_max,DOW_binary_reg_3_max,DOW_binary_reg_4_max,DOW_binary_reg_5_max,DOW_binary_reg_6_max,sin_MOY_max,cos_MOY_max,Synthetic Site Electricity Main Total Power
0,14.782966,-1.525850,-0.540707,55.112000,-1.873579,17.809999,9.395406,0.948324,-0.317305,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,100.553334
1,15.474850,-1.388764,-1.663717,54.608002,-1.918852,18.650000,8.612455,0.925541,-0.378649,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,100.063334
2,16.633362,-1.642929,-2.147240,53.978001,-2.029242,19.320000,4.503084,0.898794,-0.438371,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,100.733334
3,16.413343,-1.447732,-0.426325,52.807999,-1.979830,20.950001,3.746977,0.868199,-0.496217,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,99.921335
4,16.649153,-1.512877,-0.613491,52.034000,-2.002243,21.600000,0.000000,0.833886,-0.551937,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,99.758823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,16.008169,1.378712,-0.384731,48.074001,0.908541,24.660000,10.625756,-0.130526,0.991445,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,109.450946
514,16.985941,-0.961478,-0.831852,47.714001,-1.326790,26.340000,16.777510,0.000000,0.998135,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,108.637159
515,16.848026,-1.045253,-0.634286,47.588001,-1.507714,26.610001,11.603326,0.065403,0.992005,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,107.517187
516,16.982306,-1.165292,-0.499114,47.192001,-1.503043,26.900000,11.352782,0.130526,0.981627,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,109.115813


In [5]:
"""
Finally, we are ready to train our model!
"""
init_logging(local_results_dir=configs["exp_dir"])
run_model(configs, train_df, val_df)

INFO:92580:PID: 92580
INFO:92580:training with arch version 4
INFO:92580:Writer path: /Users/heslinge/Repos/Wattile/notebooks/exp_dir


Logging to: /Users/heslinge/Repos/Wattile/notebooks/exp_dir/output.out, PID: 92580


INFO:92580:Data transformed using minmaxscale as transformation method
INFO:92580:Available train batch factors: [1, 2, 7, 14, 37, 74, 259, 518]
INFO:92580:Requested number of batches per epoch - Train: 26, val: 1
INFO:92580:Actual number of batches per epoch - Train: 37, val: 1
INFO:92580:Number of data samples in each batch - Train: 14, val: 65
INFO:92580:Data converted to iterable dataset
INFO:92580:A new lstm RNN model instantiated
INFO:92580:Number of cores available: 8
INFO:92580:Number of logical processors available: 16
INFO:92580:Initial memory statistics (GB): {'total': 34.359738368, 'available': 15.14891264, 'percent': 55.9, 'used': 19.196289024, 'free': 0.02891776}
INFO:92580:GPU is not available for training
INFO:92580:Starting to train the model for 2 epochs!
INFO:92580:Run with arch RNN(lstm), on Synthetic Site Electricity Main Total Power, with session ID 7-9-21, is done!


In [6]:
f"""
See {exp_dir} for the results.
"""

'\nSee /Users/heslinge/Repos/Wattile/notebooks/exp_dir for the results.\n'